In [106]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from scipy import sparse
import numpy as np
import re

In [108]:
# Load your train.csv dataset
train_df = pd.read_csv('./dataset/train.csv')

In [109]:
def feature_engineering(df, sparse=0): 
    
    # Comment length
    df['length'] = df.comment_text.apply(lambda x: len(x))
    

    # Capitalization percentage
    def pct_caps(s):
        return sum([1 for c in s if c.isupper()]) / (sum(([1 for c in s if c.isalpha()])) + 1)
    df['caps'] = df.comment_text.apply(lambda x: pct_caps(x))

    # Mean Word length 
    def word_length(s):
        s = s.split(' ')
        return np.mean([len(w) for w in s if w.isalpha()])
    df['word_length'] = df.comment_text.apply(lambda x: word_length(x))

    # Average number of exclamation points 
    df['exclamation'] = df.comment_text.apply(lambda s: len([c for c in s if c == '!']))

    # Average number of question marks 
    df['question'] = df.comment_text.apply(lambda s: len([c for c in s if c == '?']))
    
    # Normalize
    for label in ['length', 'caps', 'word_length', 'question', 'exclamation']:
        minimum = df[label].min()
        diff = df[label].max() - minimum
        df[label] = df[label].apply(lambda x: (x-minimum) / (diff))

    # Strip IP Addresses
    ip = re.compile('(([2][5][0-5]\.)|([2][0-4][0-9]\.)|([0-1]?[0-9]?[0-9]\.)){3}'
                    +'(([2][5][0-5])|([2][0-4][0-9])|([0-1]?[0-9]?[0-9]))')
    def strip_ip(s, ip):
        try:
            found = ip.search(s)
            return s.replace(found.group(), ' ')
        except:
            return s

    df.comment_text = df.comment_text.apply(lambda x: strip_ip(x, ip))
    
    return df

# def merge_features(comment_text, data, engineered_features):
#     new_features = sparse.csr_matrix(data[engineered_features].values)
#     if np.isnan(new_features.data).any():
#         new_features.data = np.nan_to_num(new_features.data)
#     return sparse.hstack([comment_text, new_features])

def merge_features(comment_text, data, engineered_features):
    comment_text_sparse = TfidfVectorizer().fit_transform(data['comment_text'])  # Assuming TfidfVectorizer for comment_text
    new_features = sparse.csr_matrix(data[engineered_features].values)
    
    if np.isnan(new_features.data).any():
        new_features.data = np.nan_to_num(new_features.data)
    
    return sparse.hstack([comment_text_sparse, new_features])


In [110]:

# Assuming df is your train dataset
train_df = feature_engineering(train_df)

c:\Fyp\fypenv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Fyp\fypenv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [111]:
# Load test.csv and test_label.csv
test_df = pd.read_csv('./dataset/test.csv')
test_labels_df = pd.read_csv('./dataset/test_labels.csv')

In [113]:
# Feature engineering for test set
test_df = feature_engineering(test_df)

c:\Fyp\fypenv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Fyp\fypenv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [114]:
# Merge features for training set
engineered_features = ['length', 'caps', 'word_length', 'question', 'exclamation']
X_train = merge_features(train_df['comment_text'], train_df, engineered_features)

In [115]:
# Merge features for test set
X_test = merge_features(test_df['comment_text'], test_df, engineered_features)

In [116]:
# Convert multi-labels to a binary matrix
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer()
y_train_binary = mlb.fit_transform(train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']])

In [89]:
# Ensure X_train and y_train_binary have the same number of samples
min_samples = min(X_train.shape[0], y_train_binary.shape[0])
X_train = X_train[:min_samples]
y_train_binary = y_train_binary[:min_samples]

# Split the data
X_train, X_val, y_train_binary, y_val = train_test_split(X_train, y_train_binary, test_size=0.2, random_state=42)


In [90]:
print("X_train shape:", X_train.shape)
print("y_train_binary shape:", y_train_binary.shape)

X_train shape: (3, 189535)
y_train_binary shape: (3, 18)


In [91]:
# Check if y_train_binary has only 2 dimensions
if len(y_train_binary.shape) != 1:
    # Assuming y_train_binary is a NumPy array, extract the first column as the target
    y_train_binary = y_train_binary[:, 0]

In [92]:
# Print shapes again
print("Updated y_train_binary shape:", y_train_binary.shape)

Updated y_train_binary shape: (3,)


In [93]:
svm_model = make_pipeline(SVC(C=1, kernel='linear', probability=True))




In [94]:
print("Fitting the SVM model...")
svm_model.fit(X_train, y_train_binary)

Fitting the SVM model...


ValueError: The number of classes has to be greater than one; got 1 class

In [86]:
y_val_pred = svm_model.predict(X_val)

In [87]:
accuracy_val = np.mean(y_val_pred == y_val)
print("Validation Accuracy:", accuracy_val)

ValueError: operands could not be broadcast together with shapes (2,) (2,18) 